In [1]:
import ast
import os
import csv
from pandas import read_csv
import pandas as pd
from datetime import datetime
from decimal import Decimal
import fiona
import requests as r
import numpy as np
import pandas as pd
import getpass
import geopandas as gp
from skimage import io
import matplotlib.pyplot as plt
from osgeo import gdal
import rasterio as rio
import json
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.shutil import copy
from pathlib import Path
import pickle
import pyproj
from pyproj import Proj
import shapely.geometry
from shapely.geometry import shape
from shapely.geometry.polygon import orient
from shapely.ops import transform
import xarray as xr
from cartopy import crs
import geoviews as gv
import hvplot.xarray
import holoviews as hv
gv.extension('bokeh', 'matplotlib')
# Set Up Working Environment
inDir = os.getcwd()
os.chdir(inDir)

In [ ]:
## Soil Moisture Data ### first rearrange and calculate the weekly nanmean

In [ ]:
dir = '.../ALLRUN'

sm15 = read_csv(os.path.join(dir, 'SM', '2015SM.csv'))
sm16 = read_csv(os.path.join(dir, 'SM', '2016SM.csv'))
sm17 = read_csv(os.path.join(dir, 'SM', '2017SM.csv'))
sm1822 = read_csv(os.path.join(dir, 'SM', 'SMAPMergeDate.csv'))
smbm = read_csv(os.path.join(dir, 'SM', 'bad⁄missing.csv'))

In [ ]:
## merge all years together
small = [sm15, sm16, sm17, sm1822]
small = pd.concat(small)

newsmlist = []

for eachline in small.values:

    if eachline[2] == 0:
    
        if eachline[3] == 0:
    
            smvalue1 = 0
    
            newdict = {'County': eachline[0], 'Date': eachline[1], 'SM': smvalue1}
    
            newsmlist.append(newdict)
    
        else:
    
            smvalue2 = eachline[3]
    
            newdict = {'County': eachline[0], 'Date': eachline[1], 'SM': smvalue2}
    
            newsmlist.append(newdict)
    
    else:
    
        if eachline[3] == 0:
    
            smvalue3 = eachline[2]
    
            newdict = {'County': eachline[0], 'Date': eachline[1], 'SM': smvalue3}
    
            newsmlist.append(newdict)
    
        else:
            smvalue4 = 0.5*(eachline[2] + eachline[3])
    
            newdict = {'County': eachline[0], 'Date': eachline[1], 'SM': smvalue4}
    
            newsmlist.append(newdict)

keysdict = newsmlist[0]

with open(os.path.join(dir, 'SM',  'meanSMall.csv'), 'w') as f:  
    w = csv.DictWriter(f, keysdict.keys())
    w.writeheader()
    w.writerows(newsmlist)

In [ ]:
meanSMall = read_csv(os.path.join(dir, 'SM',  'meanSMall.csv'))

eachline = meanSMall.values[0]

import datetime

Datum = datetime.datetime.strptime(eachline[1], '%Y-%m-%d')

Datum.year

In [ ]:
### remove the zero values (missing/bad)

meanSMall = read_csv(os.path.join(dir, 'SM',  'meanSMall.csv'))

removelist = []

for eachline in meanSMall.values:

    if eachline[2] == 0:

        Datum1 = datetime.datetime.strptime(eachline[1], '%Y-%m-%d')

        DOY1 = Datum1.timetuple().tm_yday

        removedict = {'Year': Datum1.year, 'County': eachline[0], 'DOY': DOY1, 'SM': 'nan'}
    
        removelist.append(removedict)

    else:

        Datum2 = datetime.datetime.strptime(eachline[1], '%Y-%m-%d')

        DOY2 = Datum2.timetuple().tm_yday
        
        removedict = {'Year': Datum2.year, 'County': eachline[0], 'DOY': DOY2, 'SM': eachline[2]}
    
        removelist.append(removedict)

keysdict = removelist[0]

with open(os.path.join(dir, 'SM',  'RemoveSMall.csv'), 'w') as f:  
    w = csv.DictWriter(f, keysdict.keys())
    w.writeheader()
    w.writerows(removelist)    

In [ ]:
RemoveSMall = read_csv(os.path.join(dir, 'SM',  'RemoveSMall.csv'))

Yearlist = []

for eachyear in RemoveSMall.Year.unique():
    
    Yearlist.append(RemoveSMall[:][RemoveSMall.Year == eachyear])

weeklylist = []

for EachYear in Yearlist:

    year = EachYear.Year.unique()[0]

    countylist = []

    for eachcounty in EachYear.County.unique():
        
        countylist.append(EachYear[:][EachYear.County == eachcounty])

    for eachname in countylist:

        county = eachname.County.unique()

        for week in range(20, 46):

            W = "%s-W%s" % (year, week)

            import datetime
            
            ## NASS week is ISO week
            StartDate = datetime.datetime.strptime(W + '-1', "%G-W%V-%u")
            EndDate = StartDate + datetime.timedelta(days=6)
            
            SDOY = StartDate.timetuple().tm_yday
            EDOY = EndDate.timetuple().tm_yday

            singleweek = []

            for eachline in eachname.values:
            
                if SDOY <= eachline[2] <= EDOY:
            
                    singleweek.append(eachline[3])
            
            weeklyvalue = np.nanmean(singleweek)
            
            weeklydict = {'Year': year, 'County': county, 'Week': week, 'SM': weeklyvalue}
            
            weeklylist.append(weeklydict)

keysdict = weeklylist[0]

with open(os.path.join(dir, 'SM',  'WeeklySMall.csv'), 'w') as f:  
    w = csv.DictWriter(f, keysdict.keys())
    w.writeheader()
    w.writerows(weeklylist)   

In [ ]:
##############################################################################################################################

In [ ]:
###### start fill the gaps ######

In [ ]:
dir = '.../ALLRUN'

WeeklySMall = read_csv(os.path.join(dir, 'SM',  'WeeklySMall.csv'))

#colorcode = read_csv(os.path.join(dir, 'ADCcolorcode.csv'))

Yearlist = []

for eachyear in WeeklySMall.Year.unique():
    
    Yearlist.append(WeeklySMall[:][WeeklySMall.Year == eachyear])

Weeklist = []

for EY in Yearlist:
    
    for eachweek in EY.Week.unique():

        Weeklist.append(EY[:][EY.Week == eachweek])

### because some elements are empty, such as Weeklist[207]
### inappropriate to implement spatial dependence
### consider Kalman method in R package imputeTS

In [ ]:
### arrange the PHENOLOGY RATE data ###

In [ ]:
### merge the available PHENOLOGY RATE data together!!! ###

dir = '.../ALLRUN'

pr15 = read_csv(os.path.join(dir, 'Phrate',  'original', '2015Pheno.csv'))
pr15 = pr15.dropna() 

pr16 = read_csv(os.path.join(dir, 'Phrate',  'original', '2016Pheno.csv'))
pr16 = pr16.dropna() 

pr17 = read_csv(os.path.join(dir, 'Phrate',  'original', '2017Pheno.csv'))
pr17 = pr17.dropna() 

pr18 = read_csv(os.path.join(dir, 'Phrate',  'original', '2018Pheno.csv'))
pr18 = pr18.dropna() 

pr19 = read_csv(os.path.join(dir, 'Phrate',  'original', '2019Pheno.csv'))
pr19 = pr19.dropna() 

pr20 = read_csv(os.path.join(dir, 'Phrate',  'original', '2020Pheno.csv'))
pr20 = pr20.dropna() 

pr21 = read_csv(os.path.join(dir, 'Phrate',  'original', '2021Pheno.csv'))
pr21 = pr21.dropna() 

pr22 = read_csv(os.path.join(dir, 'Phrate',  'original', '2022Pheno.csv'))
pr22 = pr22.dropna() 

In [ ]:
li15 = []
li16 = []
li17 = []
li18 = []
li19 = []
li20 = []
li21 = []
li22 = []

for eachline in pr15.values:
    dict15 = {'Year': 2015, 'County': eachline[0], 'Week': eachline[1], 'PR': eachline[2]}
    li15.append(dict15)

for eachline in pr16.values:
    dict16 = {'Year': 2016, 'County': eachline[0], 'Week': eachline[1], 'PR': eachline[2]}
    li16.append(dict16)

for eachline in pr17.values:
    dict17 = {'Year': 2017, 'County': eachline[0], 'Week': eachline[1], 'PR': eachline[2]}
    li17.append(dict17)

for eachline in pr18.values:
    dict18 = {'Year': 2018, 'County': eachline[0], 'Week': eachline[1], 'PR': eachline[2]}
    li18.append(dict18)

for eachline in pr19.values:
    dict19 = {'Year': 2019, 'County': eachline[0], 'Week': eachline[1], 'PR': eachline[2]}
    li19.append(dict19)

for eachline in pr20.values:
    dict20 = {'Year': 2020, 'County': eachline[0], 'Week': eachline[1], 'PR': eachline[2]}
    li20.append(dict20)

for eachline in pr21.values:
    dict21 = {'Year': 2021, 'County': eachline[0], 'Week': eachline[1], 'PR': eachline[2]}
    li21.append(dict21)

for eachline in pr22.values:
    dict22 = {'Year': 2022, 'County': eachline[0], 'Week': eachline[1], 'PR': eachline[2]}
    li22.append(dict22)

In [ ]:
allyearlist = li15 + li16 + li17 + li18 + li19 + li20 + li21 + li22

keysdict = allyearlist[0]

with open(os.path.join(dir, 'Phrate',  'PRall.csv'), 'w') as f:  
    w = csv.DictWriter(f, keysdict.keys())
    w.writeheader()
    w.writerows(allyearlist)    

In [ ]:
### merge the available ones and the nan together ###

Avai = read_csv(os.path.join(dir, 'Phrate',  'PRall.csv'))

Miss = read_csv(os.path.join(dir, 'Phrate',  'phratemissingweeklist.csv'))

Miss

In [ ]:
frames = [Avai, Miss]

frames = pd.concat(frames)

frames = frames.sort_values(by=['Year', 'County', 'Week'])

frames

frames.to_csv(os.path.join(dir, 'Phrate', 'PRnan.csv'), na_rep='nan', index=False)

In [ ]:
### Linear interpolation for YIELD DATA
## first read the csv. of yield data

countylist = []

for eachcounty in Yield.County.unique():

    countylist.append(Yielddf[:][Yielddf.County == eachcounty])


In [ ]:
np.array(countylist[1].County.unique()[0])

In [ ]:
lengthlist = []

for countydf in countylist:
    
    lengthdict = {'County': countydf.County.unique()[0], 'Length': len(np.array(countydf.Year))}

    lengthlist.append(lengthdict)

In [ ]:
lengthlist

In [ ]:
keysdict = lengthlist[0]

directory = '.../ALLRUN'

suffix = '.csv'

with open(os.path.join(directory, 'YieldLength' + suffix), 'w') as f:  
    w = csv.DictWriter(f, keysdict.keys())
    w.writeheader()
    w.writerows(lengthlist)

In [ ]:
## create a check nan data table after deleting counties that has < 39 years

county39y = read_csv('.../ALLRUN/Y/YieldInterpolation/Yield39y.csv')

Headarray = ['County', 'Year']

Headdict = {n: n for n in Headarray} 

newlist = []

for elec in county39y.values:

    for x in range(1980, 2023):

        newlist.append(np.hstack((elec[0], x)))

withkey_dict = []

for each in newlist:
    
    withkey_dict.append(dict(zip(Headarray, each)))
    
with open(os.path.join('.../ALLRUN', 'CheckNAYield' + '.csv'), 'w') as f:  
    w = csv.DictWriter(f, Headdict.keys())
    w.writeheader()
    w.writerows(withkey_dict)

In [ ]:
## check the NA

directory = '.../ALLRUN'

suffix = '.csv'

allyeardf = read_csv(os.path.join(directory, 'CheckNAYield' + suffix))

Yielddf = read_csv('.../YIELD/19802022/Illinois19802022.csv')


In [ ]:
len(Yielddf.County.unique())

In [ ]:
countylist = []

for elemc in Yielddf.County.unique(): 
    
    countylist.append(Yielddf[:][Yielddf.County == elemc])

In [ ]:
countylist[0]

In [ ]:
for eachcoun in allyeardf.County.unique():
    
    for eachline in Yielddf.values:
        
        if eachcoun == eachline[4]:
            
            newdict = {'County': eachcoun, 'Year': eachline[0], 'Yield': }

In [ ]:
missinglist = []

b = allyeardf.Year.unique()
        
for each in countylist:
    
    a = each.Year.unique() 

    result = list((set(a) ^ set(b)))
    
    if result:

        for missyear in result:

            savedict = {'Year': missyear, 
                        'County': each.County.unique()[0], 
                        'Yield': 'nan'}
            missinglist.append(savedict)
        
keysdict = missinglist[0]

directory = '.../ALLRUN'

suffix = '.csv'

with open(os.path.join(directory, 'YieldMissing' + suffix), 'w') as f:  
    w = csv.DictWriter(f, keysdict.keys())
    w.writeheader()
    w.writerows(missinglist)

In [ ]:
############################ Interpolation #####################################

Tobedf = read_csv('.../ALLRUN/IllinoisInter19802022.csv')

Countylist = []

for each in Tobedf.County.unique():
    
    Countylist.append(Tobedf[:][Tobedf.County == each])

In [ ]:
for elem in Countylist:

    Y = pd.Series(elem.iloc[:, 2])

    elem['Yield'] = Y.interpolate()

In [ ]:
Countylist

In [ ]:
directory = '.../ALLRUN'

suffix = '.csv'

pd.concat(Countylist).to_csv(os.path.join(directory, 'Interpolated' + suffix))

In [ ]:
### open the weeklynan table

dir1 = '.../ALLRUN/nanDataTable/ANNUAL'

weeklynan = read_csv(os.path.join(dir1, 'WeeklyNA.csv'))

In [ ]:
weeklynan

In [ ]:
weeklynan = weeklynan.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]]

In [ ]:
weeklynan.iloc[:, 3:22]

In [ ]:
## select the dataframe from 2018 to 2022

df = weeklynan

year1822 = np.arange(2018, 2023, 1)

mask = df['Year'].isin(year1822)

df_1822 = df[mask]

## store the df_1821 and remove outliers in RS indices as nan values

df_1822.to_excel(os.path.join(dir1, 'df1822.xlsx'), index=False)

In [ ]:
## split the data frame by county name 

countylist = []

for eachC in df_1822.County.unique():

    countylist.append(df_1822[:][df_1822.County == eachC])

In [ ]:
from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer

In [ ]:
################# separate the whole df by county, imputation within each county #####################

countydflist = []

for dfcounty in countylist:

    dfarray = dfcounty.iloc[:, 3:22]
    
    # Setting the random_state argument for reproducibility
    
    imputer = IterativeImputer(random_state=0)
    
    imputed = imputer.fit_transform(dfarray)
    
    df_imputed = pd.DataFrame(imputed, columns=dfarray.columns)
    
    df_imputed = round(df_imputed, 6) 
    
    countydfimp = pd.concat([dfcounty.iloc[:, 0:3].reset_index(drop='TRUE'), df_imputed], axis=1)

    countydflist.append(countydfimp)

In [ ]:
impdf_1822 = pd.concat(countydflist, ignore_index=True)

In [ ]:
impdf_1822.to_csv(os.path.join(dir1, 'imp1822_county.csv'), index=False)

In [ ]:
########################## impute without separation of counties ######################## WITHOUT dealing with outliers 

from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer

dfarray = df_1822.iloc[:, 3:22]

# Setting the random_state argument for reproducibility

imputer = IterativeImputer(random_state = 0)

imputed = imputer.fit_transform(dfarray)

df_imputed = pd.DataFrame(imputed, columns = dfarray.columns)

df_imputed = round(df_imputed, 6) 

dfimp1822 = pd.concat([df_1822.iloc[:, 0:3].reset_index(drop = 'TRUE'), df_imputed], axis=1)


In [ ]:
dfimp1822.to_excel(os.path.join(dir1, 'imp1822_whole.xlsx'), index=False)

In [ ]:
######################### MICE Sklearn IterativeImpute results are bad, unreliable #############################

##################### use the smoothing to fill the gaps of SM and phrate from 2015 to 2022 #####################


In [ ]:
### open the weeklynan table

dir1 = '.../ALLRUN/nanDataTable/ANNUAL'

weeklynan = read_csv(os.path.join(dir1, 'WeeklyNA.csv'))

#weeklynan = weeklynan.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]]

## select the dataframe from 2015 to 2022

df = weeklynan

year1522 = np.arange(2015, 2023, 1)

mask = df['Year'].isin(year1522)

df_1522 = df[mask].reset_index(drop='TRUE')


In [ ]:
df_1522

In [ ]:
df_1522.to_excel(os.path.join(dir1, 'df1522.xlsx'), index=False)

In [ ]:
## split the data frame by county name 

countylist = []

for eachC in df_1522.County.unique():

    countylist.append(df_1522[:][df_1522.County == eachC])

In [ ]:
eachcounty = countylist[3]

eachcounty

In [ ]:
smArray = eachcounty.SM

eachcounty['SM'] = smArray.interpolate(method='spline', order = 3)

eachcounty.SM

In [ ]:
countyimplist = []

for eachcounty in countylist:

    smArray = eachcounty.SM
    
    eachcounty['SM'] = smArray.interpolate(method='spline', order = 3, limit_direction='both')

    prArray = eachcounty.PR

    eachcounty['PR'] = prArray.interpolate(method='polynomial', order = 2)

    countyimplist.append(eachcounty)

In [ ]:
countyimplist[1]

In [ ]:
impdf_1522 = pd.concat(countyimplist, ignore_index=True)

In [ ]:
impdf_1522

In [ ]:
impdf_1522.to_excel(os.path.join(dir1, 'df1522-SMPR.xlsx'), index=False)

In [ ]:
############################### add the data of 1980-2014 to get the imputed data table of 1980-2022

### open the weeklynan table

dir1 = '.../ALLRUN/nanDataTable/ANNUAL'

weeklynan = read_csv(os.path.join(dir1, 'WeeklyNA.csv'))

#weeklynan = weeklynan.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]]

## select the dataframe from 2015 to 2022

df = weeklynan

year8014 = np.arange(1980, 2015, 1)

mask = df['Year'].isin(year8014)

df_8014 = df[mask].reset_index(drop='TRUE')


In [ ]:
df_8014

In [ ]:
impdf_1522

In [ ]:
impdf_8022 = pd.concat([df_8014, impdf_1522], ignore_index=True)

In [ ]:
impdf_8022 = impdf_8022.iloc[:,[0,1,2,3,4,22,23,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]]

In [ ]:
impdf_8022.to_excel(os.path.join(dir1, 'impdf_8022.xlsx'), index=False)

In [8]:
######################## Fill the gaps in Yield time series #############################

dir = '.../ALLRUN'

## yield
YNANdf = read_csv(os.path.join(dir, 'Ynan.csv'))

In [9]:
countylist = []

UniqueC = YNANdf.County.unique()

for eachcounty in UniqueC:
    
    countylist.append(YNANdf[:][YNANdf.County == eachcounty])

In [10]:
countylist[13]

,Year,County,Yield
559,1980,COLES,37.0
560,1981,COLES,41.0
561,1982,COLES,39.5
562,1983,COLES,28.0
563,1984,COLES,31.0
564,1985,COLES,49.0
565,1986,COLES,45.5
566,1987,COLES,40.0
567,1988,COLES,28.0
568,1989,COLES,41.0


In [11]:
impY = []

for eachcounty in countylist:

    yArray = eachcounty.Yield
    
    eachcounty['Yield'] = yArray.interpolate(method='linear')

    impY.append(eachcounty)

In [12]:
impY

[    Year County  Yield
 0   1980  ADAMS   32.5
 1   1981  ADAMS   36.0
 2   1982  ADAMS   37.0
 3   1983  ADAMS   23.0
 4   1984  ADAMS   28.5
 5   1985  ADAMS   39.0
 6   1986  ADAMS   36.5
 7   1987  ADAMS   37.0
 8   1988  ADAMS   27.0
 9   1989  ADAMS   29.0
 10  1990  ADAMS   33.5
 11  1991  ADAMS   36.5
 12  1992  ADAMS   38.5
 13  1993  ADAMS   40.0
 14  1994  ADAMS   46.0
 15  1995  ADAMS   38.0
 16  1996  ADAMS   40.0
 17  1997  ADAMS   45.0
 18  1998  ADAMS   41.0
 19  1999  ADAMS   35.0
 20  2000  ADAMS   47.0
 21  2001  ADAMS   44.0
 22  2002  ADAMS   48.0
 23  2003  ADAMS   44.0
 24  2004  ADAMS   49.0
 25  2005  ADAMS   43.0
 26  2006  ADAMS   43.0
 27  2007  ADAMS   37.0
 28  2008  ADAMS   41.0
 29  2009  ADAMS   47.0
 30  2010  ADAMS   48.7
 31  2011  ADAMS   41.4
 32  2012  ADAMS   40.7
 33  2013  ADAMS   43.6
 34  2014  ADAMS   50.7
 35  2015  ADAMS   52.1
 36  2016  ADAMS   56.8
 37  2017  ADAMS   55.9
 38  2018  ADAMS   59.3
 39  2019  ADAMS   49.8
 40  2020  ADAMS

In [14]:
impY = pd.concat(impY, ignore_index=True)

In [15]:
impY.to_excel(os.path.join(dir, 'impY.xlsx'), index=False)